In [2]:
# Data transformation
library(dplyr)
library(readr)
library(lubridate)
# Data visualization
library(ggplot2)
library(scales)
# Custom R file
source("../Common/func.R")

In [17]:
# Whole House Water
whwDF <-    read_csv("../../data/Water_WHW.csv", show_col_types = FALSE) %>%
            convert_timeStamp("unix_ts")
# Whole House Electricity
wheDF <-    read_csv("../../data/Electricity_WHE.csv", show_col_types = FALSE) %>%
            convert_timeStamp("unix_ts") %>%
            mutate(avg_rate_modif = if_else(is.na(St - lag(St)), 0, St - lag(St)))
# Whole House Natural Gas
whgDF <-    read_csv("../../data/NaturalGas_WHG.csv", show_col_types = FALSE) %>%
            convert_timeStamp("unix_ts") %>%
            mutate(avg_rate_modif = avg_rate / 60)

ERROR: Error in eval_tidy(pair$lhs, env = default_env): objet 'x' introuvable


In [18]:
# Whole House Water
whwDaily <- whwDF %>%
    group_by(date) %>% 
    summarise(
        year        = factor(first(year)),
        month       = factor(first(month)),
        day         = factor(first(day)),
        wDay        = first(wday(date, week_start = 1)),
        consumption = sum(avg_rate) / 1000 # Liters to kilo Liters
    )
# Whole House Electricity
wheDaily <- wheDF %>%
    group_by(date) %>% 
    summarise(
        year        = factor(first(year)),
        month       = factor(first(month)),
        day         = factor(first(day)),
        wDay        = first(wday(date, week_start = 1)),
        consumption = sum(avg_rate_modif) / 1000 # Watts per hour to kWh
    )
# Whole House Natural Gas
whgDaily <- whgDF %>%
    group_by(date) %>% 
    summarise(
        year        = factor(first(year)),
        month       = factor(first(month)),
        day         = factor(first(day)),
        wDay        = first(wday(date, week_start = 1)),
        consumption = sum(avg_rate) # meter cube
    )

In [46]:
all_metrics <- whwDaily %>%
    left_join(wheDaily, by = c("date"), suffix = c("_water", "_elec")) %>%
    left_join(whgDaily, by = c("date"), suffix = c("", "_gas")) %>%
    select(date, year_water, month_water, day_water, wDay_water, consumption_water, consumption_elec, consumption) %>%
    rename(
        year                = year_water,
        month               = month_water,
        day                 = day_water,
        wDay                = wDay_water,
        `consumption (kL)`  = consumption_water,
        `consumption (kWh)` = consumption_elec,
        `consumption (m3)`  = consumption
    )

In [55]:
lmodel = lm(log10(all_metrics$`consumption (kWh)`) ~ log10(all_metrics$`consumption (kL)`) + log10(all_metrics$`consumption (m3)`), data = all_metrics)
summary(lmodel)

ERROR: Error in lm.fit(x, y, offset = offset, singular.ok = singular.ok, ...): NA/NaN/Inf dans 'x'


In [56]:
cor_w_e <- cor(
    x = log(all_metrics$`consumption (kWh)`), 
    y = log(all_metrics$`consumption (m3)`), 
    use = "everything", method = c("pearson")
)
print(cor_w_e)

[1] NaN
